# タスク 3: 質問応答に Amazon Bedrock を使用する

このノートブックでは、関連する完全なコンテキストを含むリクエストをモデルに送信し、応答が返されるのを待つことで、クエリに対する情報の応答をAmazon Bedrockを通じて Titan モデルから得る方法を学習します。これにより、事前にドキュメントを準備してインデックスを作成することなく、モデルが質問に対して事実に基づいた応答を返すという課題に対処します。

このノートブックは、**Retrieval-Augmented Generation (RAG)** が行うことをシミュレートしますが、実際には RAG を使用しません。このアプローチは、短いドキュメントまたはシングルトンアプリケーションで機能します。モデルに送信されるプロンプトに収まらないような大規模なエンタープライズドキュメントを使用するエンタープライズレベルの質問応答には拡張できない可能性があります。

**質問応答 (QA)** は、自然言語で提示された事実に基づくクエリに対する応答を抽出する重要なタスクです。通常、QA システムは、構造化データまたは非構造化データを含むナレッジ ベースに対してクエリを処理し、正確な情報を含む応答を生成します。高い精度を確保することは、特にエンタープライズ ユース ケースにおいて、有用で信頼性が高く、信頼できる質問応答システムを開発する上で重要です。

## シナリオ

AnyCompany で、同社が製造する特定の車両モデルのタイヤ交換に関する情報を提供するよう質問応答モデルに求める状況をモデル化してみます。まず、「ゼロショット」アプローチを使用してモデルにクエリを実行し、トレーニングデータのみに基づいて適切な回答を提供できるかどうかを確認します。

ただし、偽の車両モデルを試して同様の応答が得られた場合、モデルがより一般的な回答を「幻覚」しているように見えることがわかります。これは、各モデルのタイヤの詳細を提供するために、Example Company の実際の車両マニュアルを使用してモデルのトレーニングを拡張する必要があることを意味します。

このラボでは、外部データなしでこのような「検索拡張生成」(RAG) アプローチをシミュレートします。AnyCompany Model Z 車両のタイヤ交換方法を説明した詳細なマニュアルの抜粋を提供します。このコンテキスト内のサンプル コンテンツを活用して、モデルがカスタマイズされた正確な回答を提供できるかどうかをテストします。

## Task 3.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## タスク 3.2: モデルの知識を使用した Q&A
このセクションでは、Bedrock サービスによって提供されるモデルを使用して、トレーニングフェーズ中に獲得した知識に基づいて質問に答えます。

このタスクでは、Amazon Bedrock クライアントのinvoke_model() メソッドを使用します。このメソッドを使用するために必要な必須パラメータは、Amazon Bedrock モデル ARN を表す modelId と、タスクのプロンプトである body です。

body プロンプトは、選択した基盤モデル プロバイダーに応じて変わります。これについては、以下で詳しく説明します。

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Bedrock サービスによって提供されるモデルを使用して、トレーニング フェーズ中に得られた知識に基づいて質問に答えます。

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

## タスク3.3: JSON本体を渡してモデルを呼び出し、レスポンスを生成する

In [ ]:
#model configuration
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
try:
    
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("results")[0].get("outputText")
    print(answer.strip())

except botocore.exceptions.ClientError as error:
    if  error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error


モデルは車のパンクしたタイヤを交換する手順を概説した回答を出しますが、同じ説明はどの車にも当てはまります。残念ながら、これはスペアタイヤのない AnyCompany AC8 には正しい回答ではありません。これは、モデルが車のタイヤ交換手順を含むデータでトレーニングされているためです。

この問題の別の例は、Amazon Tirana など、完全に偽の車のブランドとモデルに同じ質問をしてみることでわかります。

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, 
                   "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

プロンプトの質問では、モデルは現実的な回答を提供できません。

生成 AI モデルが特定の車種に有効な指示に基づいて回答を提供するために、プロンプトの一部として追加の知識ベースを提供することで、モデルの知識をオンザフライで拡張できます。

これを使用してアプリケーションを改善する方法を見てみましょう。

以下は、AnyCompany AC8 のマニュアルからの抜粋であると仮定します (実際に実在するマニュアルではありませんが、そのように扱います)。 このドキュメントは、Titan Large コンテキスト ウィンドウに完全に収まるほど短く便利です。

```plain
Tires and Tire Pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to Find Recommended Tire Pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the Tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for Using the Tire Mobility Kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Rivesla or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible.
```

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### ここで、抜粋全体を質問とともにモデルに渡します。

In [ ]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

### タスク 3.4: boto3 経由でモデルを呼び出してレスポンスを生成する

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

モデルがコンテキストを理解して適切な回答を生成するのに時間がかかるため、応答を数秒間待たなければならなくなり、ユーザーエクスペリエンスが低下する可能性があります。

Bedrock は、モデルがトークンを生成するとサービスが出力を生成するストリーミング機能もサポートしています。これを実装する方法の例を次に示します。

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
# response with stream
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

応答には、タイヤの交換方法に関する要約された手順が示されています。

これで、検索拡張生成 (RAG) または拡張プロセスを活用して、提供された特定のコンテキストと情報に合わせて調整された応答を生成する方法を学習しました。

### 試してみましょう
- 特定のユースケースに合わせてプロンプトを変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えることで、サービスのレイテンシと応答性がどのように変化するかを理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を取得します。

### クリーンアップ

あなたはこのノートブックを完了しました。ラボの次のパートに移るには、下記を実行してください。:

- このノートブックファイルを閉じ、**タスク 4** に進んでください。